# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.3129998275024488),
 (1, 1.3129998275024488),
 (2, 1.3129998275024488),
 (3, 1.3129998275024488),
 (4, 1.3129998275024488)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('what', 10)]),
 (1, [('is', 18), ('it', 18)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.027043536525846346),
   (None, 0.031550205162345524),
   (None, 0.03301519833156441),
   (None, 0.060173015101643634),
   (None, 0.10072221693269612),
   (None, 0.10389517401411474),
   (None, 0.12211864567257102),
   (None, 0.15435472844083908),
   (None, 0.22169500318402546),
   (None, 0.2724300124028082),
   (None, 0.305485386743935),
   (None, 0.30823738760307473),
   (None, 0.33444599398857844),
   (None, 0.3652540036120818),
   (None, 0.3653169892589593),
   (None, 0.39742823002738714),
   (None, 0.4403620113019441),
   (None, 0.4631432820728105),
   (None, 0.4659703134759545)]),
 (1,
  [(None, 0.5469742053569722),
   (None, 0.6310485979498993),
   (None, 0.6331201098213773),
   (None, 0.7244314502374273),
   (None, 0.8338466332244877),
   (None, 0.8419214097544389),
   (None, 0.8548277759561446),
   (None, 0.9032025035878632),
   (None, 0.9607144447103095),
   (None, 0.962841832518602),
   (None, 0.9639480436948871)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
from typing import List, Tuple
import numpy as np

import random
from functools import reduce


# Функция RECORDREADER для генерации случайных чисел в заданном диапазоне
def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

# Генерация списка случайных чисел
record = RECORDREADER(15)
print(record)

# Генерация списка случайных чисел
record = RECORDREADER(15)
print(record)


# MAP: Каждое число получает ключ "max".
def map_max(numbers: List[int]) -> List[Tuple[str, int]]:
    return [("max", num) for num in numbers]

# REDUCE: Выбирает наибольшее из всех значений.
def reduce_max(mapped_values: List[Tuple[str, int]]) -> Tuple[str, int]:
    max_value = max(value for key, value in mapped_values)
    return ("max", max_value)

# Запуск MapReduce
mapped = map_max(record)
result = reduce_max(mapped)
print(result)



[18, 37, 57, 10, 81, 11, 40, 50, 76, 85, 99, 67, 0, 87, 18]
[0, 10, 94, 72, 34, 33, 88, 37, 95, 41, 76, 30, 78, 62, 2]
('max', 95)


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
# Функция RECORDREADER для генерации случайных чисел в заданном диапазоне
def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

def MAP(value: int) -> Iterator[Tuple[str, Tuple[int, int]]]:
    """Передаёт каждое число с ключом 'sum_count'."""
    yield ("sum_count", (value, 1))


def REDUCE(key: str, values: Iterator[Tuple[int, int]]) -> Iterator[Tuple[str, float]]:
    """Вычисляет среднее значение."""
    total_sum, total_count = 0, 0
    for val, count in values:
        total_sum += val
        total_count += count
    yield (key, total_sum / total_count if total_count else 0)

# Пример выполнения
data = RECORDREADER(10)
mapped = (pair for value in data for pair in MAP(value))
grouped = {"sum_count": [(val, cnt) for _, (val, cnt) in mapped]}
result = list(REDUCE("sum_count", iter(grouped["sum_count"])))
print(data)
print(result)  # [('sum_count', <среднее значение>)]


[78, 62, 91, 74, 93, 38, 74, 91, 60, 16]
[('sum_count', 67.7)]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
def RECORDREADER(size: int) -> Iterator[Tuple[str, int]]:
    """Генерирует случайные пары (ключ, число)."""
    keys = ["A", "B", "C", "D"]
    data = [(random.choice(keys), random.randint(1, 100)) for _ in range(size)]
    for item in data:
        print('input ', item)
        yield item


def MAP(pair: Tuple[str, int]) -> Iterator[Tuple[str, int]]:
    """Пропускает данные без изменений."""
    yield pair

def REDUCE(key: str, values: Iterator[int]) -> Iterator[Tuple[str, List[int]]]:
    """Группирует значения по ключу с сортировкой."""
    yield (key, sorted(values))

# Пример выполнения 1
data = RECORDREADER(10) # Генерация 10 чисел

mapped = (pair for value in data for pair in MAP(value))
grouped = {}
for k, v in mapped:
    grouped.setdefault(k, []).append(v)
result = sorted([item for key, values in grouped.items() for item in REDUCE(key, iter(values))])
print('result 1', result)  # [('A', [...]), ('B', [...]), ('C', [...])]

print('------------------------')

#Пример выполнения 2
data = RECORDREADER(15) # Генерация 10 чисел
mapped = (pair for value in data for pair in MAP(value))
grouped = {}
for k, v in mapped:
    grouped.setdefault(k, []).append(v)
result = sorted([item for key, values in grouped.items() for item in REDUCE(key, iter(values))])
print('result 2', result)  # [('A', [...]), ('B', [...]), ('C', [...])]


input  ('B', 64)
input  ('A', 93)
input  ('C', 46)
input  ('C', 51)
input  ('B', 62)
input  ('C', 29)
input  ('A', 90)
input  ('D', 37)
input  ('A', 68)
input  ('A', 87)
result 1 [('A', [68, 87, 90, 93]), ('B', [62, 64]), ('C', [29, 46, 51]), ('D', [37])]
------------------------
input  ('A', 47)
input  ('C', 3)
input  ('C', 18)
input  ('A', 49)
input  ('B', 62)
input  ('B', 54)
input  ('A', 46)
input  ('C', 26)
input  ('A', 1)
input  ('B', 11)
input  ('D', 100)
input  ('A', 87)
input  ('A', 25)
input  ('D', 70)
input  ('D', 52)
result 2 [('A', [1, 25, 46, 47, 49, 87]), ('B', [11, 54, 62]), ('C', [3, 18, 26]), ('D', [52, 70, 100])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
def RECORDREADER(size: int, min_val: int = 0, max_val: int = 100) -> Iterator[int]:
    """Генерирует случайный список чисел."""
    data = [random.randint(min_val, max_val) for _ in range(size)]
    for num in data:
        print('generated val ', num)
        yield num



def MAP(value: int) -> Iterator[Tuple[int, None]]:
    """Отмечает каждое число как встреченное."""
    yield (value, None)


def REDUCE(key: int, _: Iterator[None]) -> Iterator[int]:
    """Выдаёт уникальные ключи."""
    yield key

# Пример выполнения
data = RECORDREADER(15, 1, 10)  # Генерация 15-ти чисел от 1 до 10 (чтобы были дубликаты)
mapped = (pair for value in data for pair in MAP(value))
grouped = {k: None for k, _ in mapped}  # Убираем дубликаты
print(' ----- ')
result = sorted(k for k in grouped)
print("Unique values:", result)


generated val  9
generated val  4
generated val  7
generated val  8
generated val  2
generated val  8
generated val  1
generated val  10
generated val  2
generated val  3
generated val  5
generated val  6
generated val  9
generated val  1
generated val  5
 ----- 
Unique values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
from collections import defaultdict

# Функция для фильтрации по предикату C
def MAP_selection(el_list, predicate):
    mapped_result = []
    for t in el_list:
        if predicate(t):  # Применение предиката
            mapped_result.append((t, t))  # Кортеж (t, t), если предикат верен
    return mapped_result

# Функция Reduce, которая возвращает исходный кортеж
def REDUCE_selection(mapped_items):
    return [key for key, _ in mapped_items]  # Просто возвращаем ключи (они равны значениям)

# Предикат для выборки (например, проверка на четность первого элемента)
def C(t):
    return t[0] % 2 == 0  # Проверка, что первый элемент четный

# Генерация данных
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

# Пример использования
data = RECORDREADER(10)
mapped = MAP_selection(data, C)  # Применяем MAP, чтобы оставить только те кортежи, где первый элемент четный
result = REDUCE_selection(mapped)  # Переходим к REDUCE, где просто выводим все подходящие кортежи
print(result)


[(20, 83), (78, 34), (48, 1), (74, 78), (48, 78)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [80]:
# Функция MAP для проекции
def MAP_projection(el_list, attributes_to_keep):
    mapped_result = []
    for t in el_list:
        t_prime = tuple(val for i, val in enumerate(t) if i in attributes_to_keep)  # Оставляем только нужные атрибуты
        mapped_result.append((t_prime, t_prime))  # Пара (t', t')
    return mapped_result

# Функция Reduce, которая возвращает только один уникальный кортеж для каждого ключа
def REDUCE_projection(mapped_items):
    return [key for key, _ in mapped_items]  # Просто возвращаем ключи (они равны значениям)

# Пример использования
attributes_to_keep = {0, 2}  # Например, хотим оставить только первый и третий атрибут
data = RECORDREADER(10)
mapped = MAP_projection(data, attributes_to_keep)  # Применяем MAP, чтобы оставить только нужные атрибуты
result = REDUCE_projection(mapped)  # Переходим к REDUCE, где просто выводим уникальные проекционные кортежи
print(result)


[(96, 1), (22, 70), (75, 22), (14, 24), (46, 55), (83, 9), (75, 60), (81, 49), (70, 28), (12, 20)]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [79]:
import random
from typing import Iterator

# Функция MAP для преобразования каждого входного элемента t в пару (t, t)
def MAP(t):
    return (t, t)  # Возвращаем пару (t, t) для каждого элемента

# Функция REDUCE для обработки (t, t) пар
def REDUCE(key, values: Iterator):
    return (key, key)  # Для каждого ключа возвращаем пару (key, key)

# Генератор случайных данных
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]  # Генерация кортежей с случайными числами

# Группировка значений по ключу
def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебор элементов
        t[k2] = t.get(k2, []) + [v2]  # Группируем элементы по ключу
    return t.items()  # Возвращаем список кортежей (ключ, значения)

# Пример использования
record = RECORDREADER(5)  # Генерация списка кортежей
print("Record:", record)

# Применение функции MAP ко всем элементам
map_output = list(map(lambda x: MAP(x), record))
print("MAP output:", map_output)

# Группировка значений по ключу
shuffle_output = group_by_key(map_output)  # Группируем по ключу
shuffle_output = list(shuffle_output)  # Преобразуем в список
print("Shuffle output:", shuffle_output)

# Применение функции REDUCE к результату группировки
reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print("Reduce output:", reduce_output)

# Функция MAP для объединения элементов в пары (t, t)
def MAP_union(el_list):
    return [(t, t) for t in el_list]  # Для каждого элемента создаем пару (t, t)

# Функция REDUCE для обработки пар (t, t)
def REDUCE_union(mapped_items):
    return [key for key, _ in mapped_items]  # Возвращаем только ключи

# Пример использования для MAP_union и REDUCE_union
data = RECORDREADER(10)  # Генерация данных
mapped = MAP_union(data)  # Применяем MAP, создавая пары (t, t)
result = REDUCE_union(mapped)  # Применяем REDUCE для сбора всех ключей
print("Reduce Union Result:", result)


Record: [(30, 28, 63), (52, 39, 60), (57, 10, 16), (49, 84, 38), (75, 66, 47)]
MAP output: [((30, 28, 63), (30, 28, 63)), ((52, 39, 60), (52, 39, 60)), ((57, 10, 16), (57, 10, 16)), ((49, 84, 38), (49, 84, 38)), ((75, 66, 47), (75, 66, 47))]
Shuffle output: [((30, 28, 63), [(30, 28, 63)]), ((52, 39, 60), [(52, 39, 60)]), ((57, 10, 16), [(57, 10, 16)]), ((49, 84, 38), [(49, 84, 38)]), ((75, 66, 47), [(75, 66, 47)])]
Reduce output: [(30, 28, 63), (52, 39, 60), (57, 10, 16), (49, 84, 38), (75, 66, 47)]
Reduce Union Result: [(29, 97, 77), (93, 21, 85), (12, 15, 61), (1, 2, 31), (95, 55, 2), (18, 56, 18), (1, 99, 23), (28, 1, 87), (28, 44, 61), (98, 26, 84)]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [70]:
import random
from collections import defaultdict

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]  # Генерация случайных кортежей

def MAP_intersection(el_list):
    return [(t, t) for t in el_list]  # Преобразуем каждый кортеж в пару (t, t)

def group_by_key(iterable):
    grouped = defaultdict(list)
    for k, v in iterable:
        grouped[k].append(v)
    return grouped.items()

def REDUCE_intersection(grouped_items):
    return [(key, key) for key, values in grouped_items if len(values) == 2]  # Пересечение множеств

# Пример использования
data1 = RECORDREADER(10)
data2 = RECORDREADER(10)
print("Dataset 1:", data1)
print("Dataset 2:", data2)

mapped1 = MAP_intersection(data1)
mapped2 = MAP_intersection(data2)
mapped_all = mapped1 + mapped2  # Объединяем два набора

grouped_data = group_by_key(mapped_all)
intersection_result = REDUCE_intersection(grouped_data)

print("Intersection:", intersection_result)


Dataset 1: [(0, 0), (3, 3), (2, 2), (2, 0), (3, 2), (1, 1), (0, 0), (2, 2), (1, 1), (1, 1)]
Dataset 2: [(3, 2), (2, 2), (3, 1), (0, 1), (0, 0), (1, 2), (1, 0), (2, 0), (2, 0), (3, 1)]
Intersection: [((3, 2), (3, 2)), ((3, 1), (3, 1))]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [78]:
import random
from collections import defaultdict
from typing import List, Tuple as T, Iterator

rels = [1, 2]

class DataTuple:
    # Класс для хранения данных и id представления
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count: int) -> DataTuple:
    # Генерация случайного кортежа с count парами чисел и случайным rel_id
    data = tuple([(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)])
    rel_id = random.choice(rels)  # Выбор случайного id представления из массива rels
    return DataTuple(data, rel_id)

def RECORDREADER(count: int) -> List[DataTuple]:
    # Генерация списка случайных кортежей данных
    return [get_random_tuple(3) for _ in range(count)]

def MAP(t: DataTuple) -> T[tuple, int]:
    # Функция MAP, которая возвращает кортеж (данные, id)
    return (t.data, t.rel_id)

def REDUCE(key: int, values: Iterator[int]) -> T[int, int]:
    # Функция REDUCE, которая оставляет только элементы с rel_id == 1
    if values == [rels[0]]:
        return key, key
    return None

def group_by_key(iterable: Iterator[T[tuple, int]]) -> list:
    # Группировка значений по ключу
    grouped = defaultdict(list)
    for key, value in iterable:
        grouped[key].append(value)
    return list(grouped.items())

# Генерация данных
record = RECORDREADER(100)

# Применение функции MAP к данным
map_output = list(map(MAP, record))
print("MAP output:", map_output)

# Группировка значений по ключу
grouped_output = group_by_key(map_output)
print("Grouped output:", grouped_output)

# Применение функции REDUCE к сгруппированным данным
reduce_output = [el[0] for el in map(lambda x: REDUCE(*x), grouped_output) if el is not None]
print("Reduce output:", reduce_output)


MAP output: [(((0, 3), (1, 1), (3, 1)), 1), (((0, 1), (1, 0), (2, 1)), 1), (((0, 2), (0, 0), (1, 0)), 1), (((0, 2), (3, 3), (2, 1)), 1), (((0, 1), (2, 2), (1, 3)), 2), (((1, 2), (0, 3), (1, 1)), 1), (((1, 3), (2, 2), (1, 2)), 2), (((1, 1), (1, 0), (0, 1)), 2), (((0, 2), (0, 3), (0, 2)), 1), (((1, 2), (0, 0), (1, 0)), 1), (((0, 1), (1, 0), (2, 3)), 1), (((3, 1), (0, 3), (2, 1)), 1), (((2, 0), (2, 3), (0, 3)), 2), (((0, 2), (0, 0), (0, 1)), 2), (((2, 2), (2, 0), (3, 0)), 2), (((2, 3), (1, 0), (1, 2)), 2), (((1, 1), (0, 1), (1, 2)), 2), (((1, 2), (3, 3), (3, 0)), 1), (((2, 0), (3, 1), (1, 3)), 1), (((0, 2), (1, 2), (3, 3)), 2), (((0, 1), (3, 2), (2, 2)), 1), (((1, 3), (2, 2), (3, 2)), 2), (((0, 3), (0, 3), (0, 1)), 2), (((3, 2), (2, 2), (2, 3)), 2), (((2, 1), (0, 0), (3, 1)), 1), (((1, 3), (2, 0), (2, 1)), 2), (((2, 3), (1, 0), (2, 0)), 2), (((2, 2), (3, 1), (2, 0)), 2), (((1, 2), (1, 3), (0, 1)), 1), (((0, 3), (2, 1), (1, 3)), 2), (((2, 0), (1, 1), (0, 2)), 2), (((1, 2), (2, 2), (1, 1)),

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
from collections import defaultdict

# Функция MAP для естественного соединения
def MAP_R(t):
    return (t[1], ('R', t[0]))  # Для отношения R (a, b) создаем пару (b, (R, a))

def MAP_S(t):
    return (t[0], ('S', t[1]))  # Для отношения S (b, c) создаем пару (b, (S, c))

# Функция REDUCE для соединения по ключу b
def REDUCE(key, values):
    result = []
    R_values = [v[1] for v in values if v[0] == 'R']  # Извлекаем все значения из R
    S_values = [v[1] for v in values if v[0] == 'S']  # Извлекаем все значения из S
    # Создаем все возможные комбинации (a, b, c)
    for a in R_values:
        for c in S_values:
            result.append((a, key, c))  # Формируем тройку (a, b, c)
    return result

# Пример данных
R = [(1, 'a'), (2, 'b'), (3, 'a')]
S = [('a', 10), ('b', 20), ('a', 30)]

# Применение Map и Reduce
map_output_R = list(map(MAP_R, R))
map_output_S = list(map(MAP_S, S))

# Группировка по ключу
grouped = defaultdict(list)
for pair in map_output_R + map_output_S:
    grouped[pair[0]].append(pair[1])

# Применение Reduce
reduce_output = []
for key, values in grouped.items():
    reduce_output.extend(REDUCE(key, values))

print(reduce_output)


[(1, 'a', 10), (1, 'a', 30), (3, 'a', 10), (3, 'a', 30), (2, 'b', 20)]


**Как работает алгоритм:**

На этапе Map мы генерируем пары ключ-значение для обоих наборов данных R и S.
На этапе Reduce для каждого ключа b мы собираем все возможные комбинации (𝑎,𝑏,𝑐)(a,b,c), если b присутствует и в R, и в S.
Вывод: результатом будет множество троек, где b совпадает в обоих отношениях.

**Map Function:** Для каждого кортежа (a,b)(a, b)(a,b) из отношения RRR создается пара (b,(R,a))(b, (R, a))(b,(R,a)). Для каждого кортежа (b,c)(b, c)(b,c) из отношения SSS создается пара (b,(S,c))(b, (S, c))(b,(S,c)). Это позволяет группировать все элементы по ключу bbb, который будет использоваться для соединения.

**Reduce Function**: Каждый ключ bbb будет ассоциирован со списком пар. Эти пары могут быть как (R,a)(R, a)(R,a), так и (S,c)(S, c)(S,c). В процессе редукции нужно перебрать все пары и создать тройки вида (a,b,c)(a, b, c)(a,b,c), используя пары из RRR и SSS.



### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
# Функция MAP для группировки
def MAP(t):
    return (t[0], t[1])  # Возвращаем (a, b) для кортежа (a, b, c)

# Функция REDUCE для агрегации
def REDUCE(key, values):
    return (key, sum(values))  # Применение агрегирующей операции SUM

# Пример данных
data = [(1, 10, 100), (1, 20, 200), (2, 30, 300), (2, 40, 400)]

# Применение Map
map_output = list(map(MAP, data))

# Группировка по ключу
grouped = defaultdict(list)
for key, value in map_output:
    grouped[key].append(value)

# Применение Reduce
reduce_output = []
for key, values in grouped.items():
    reduce_output.append(REDUCE(key, values))

print(reduce_output)


[(1, 30), (2, 70)]


Как работает алгоритм:
На этапе Map для каждого кортежа создаем пару (a,b)(a, b)(a,b), где aaa — ключ, а bbb — значение.

На этапе Reduce для каждого ключа aaa мы агрегируем все значения bbb с помощью функции, например, суммы θ=SUMθ = \text{SUM}θ=SUM.

Вывод: результатом будет список кортежей (a,x)(a, x)(a,x), где xxx — результат применения агрегирующей операции.

Map Function: Для каждого кортежа (a,b,c)(a, b, c)(a,b,c) создается пара (a,b)(a, b)(a,b), где aaa — это ключ, а bbb — значение, которое нужно агрегировать.

Reduce Function: Для каждого ключа aaa мы собираем список значений bbb, ассоциированных с ним, и применяем агрегирующую функцию θθθ, например, SUM или MAX.


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
# Функция MAP для умножения матрицы на вектор
def MAP(matrix_row, vector):
    result = []
    for i, value in enumerate(matrix_row):
        result.append((i, value * vector[i]))  # Умножаем каждый элемент строки матрицы на соответствующий элемент вектора
    return result

# Функция REDUCE для суммирования произведений
def REDUCE(key, values):
    return sum(values)  # Суммируем все произведения для каждого индекса

# Пример данных
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]  # Матрица
vector = [1, 2, 3]  # Вектор

# Применение Map
map_output = []
for row in matrix:
    map_output.extend(MAP(row, vector))

# Группировка по ключу
grouped = defaultdict(list)
for key, value in map_output:
    grouped[key].append(value)

# Применение Reduce
reduce_output = []
for key, values in grouped.items():
    reduce_output.append(REDUCE(key, values))

print(reduce_output)


[12, 30, 54]


Как работает алгоритм:

На этапе Map для каждой строки матрицы мы умножаем каждый элемент строки на соответствующий элемент вектора.
На этапе Reduce для каждого индекса строки матрицы мы суммируем все произведения, чтобы получить итоговое значение.
Вывод: результат умножения матрицы на вектор.



## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [81]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))


Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [82]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)


Проверьте своё решение

In [83]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [84]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [62]:
import numpy as np

# Определяем размеры матриц
I, J, K = 2, 3, 40  # Размеры матриц: I x J и J x K

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Первая матрица I x J
big_mat = np.random.rand(J, K)  # Вторая матрица J x K

# Эталонное решение с использованием numpy
reference_solution = np.matmul(small_mat, big_mat)

# Функция чтения матриц и преобразования их в (ключ, значение)
def read_matrices():
    for i in range(I):
        for j in range(J):
            yield (0, i, j), small_mat[i, j]  # Первый элемент

    for j in range(J):
        for k in range(K):
            yield (1, j, k), big_mat[j, k]  # Второй элемент

# MAP-функция: разделение данных для распределенной обработки
def map_join(key, value):
    mat_id, i, j = key
    if mat_id == 0:
        yield j, (0, i, value)  # Распространяем элемент первой матрицы по столбцу
    else:
        yield i, (1, j, value)  # Распространяем элемент второй матрицы по строке

# REDUCE-функция: объединение элементов и вычисление их произведений
def reduce_join(_, values):
    first_mat = [v for v in values if v[0] == 0]
    second_mat = [v for v in values if v[0] == 1]

    for f in first_mat:
        for s in second_mat:
            yield (f[1], s[1]), f[2] * s[2]  # Перемножение значений

# MAP-функция: передача произведений дальше
def map_sum(key, value):
    yield key, value

# REDUCE-функция: суммирование произведений для получения итоговых значений
def reduce_sum(key, values):
    yield key, sum(values)  # Суммируем элементы с одинаковыми ключами

# Универсальная функция для запуска MapReduce-процессов
def run_mapreduce(data_reader, map_func, reduce_func):
    mapped = (map_func(k, v) for k, v in data_reader())
    grouped = {}
    for k, v in (pair for sublist in mapped for pair in sublist):
        grouped.setdefault(k, []).append(v)
    return (pair for sublist in (reduce_func(k, v) for k, v in grouped.items()) for pair in sublist)

# Запуск первого MapReduce (JOIN)
joined = list(run_mapreduce(read_matrices, map_join, reduce_join))

# Запуск второго MapReduce (умножение и суммирование)
solution = run_mapreduce(lambda: joined, map_sum, reduce_sum)

# Функция преобразования списка пар в матрицу
def to_matrix(reduce_output):
    result = np.zeros((I, K))  # Инициализация нулевой матрицы
    for (i, k), value in reduce_output:
        result[i, k] = value  # Заполняем результат
    return result

# Проверка совпадения результатов
print(np.allclose(reference_solution, to_matrix(solution)))


True


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
import numpy as np

# Определяем размеры матриц
I = 2
J = 3
K = 4 * 10

# Создаем случайные матрицы
small_mat = np.random.rand(I, J)  # Матрица I x J
big_mat = np.random.rand(J, K)    # Матрица J x K

# Вычисляем эталонное решение с помощью стандартного умножения матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция, превращающая вложенный итерируемый объект в плоский список
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

# Группировка значений по ключу
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

# Распределенная группировка значений по ключу с разбиением на партиции
def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]

    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]  # Определяем, в какую партицию попадет ключ
            p[k2] = p.get(k2, []) + [v2]

    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in enumerate(partitions)]

# Функция распределения ключей по партициям
def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

# Основная функция MapReduce
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    # Фаза Map: применяем MAP к входным данным
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    # Агрегируем данные по ключам
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    # Вывод количества переданных пар ключ-значение
    print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))

    return reduce_outputs

# Функция преобразования результата в матрицу
def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output) + 1
    K = max(k for ((i,k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i, k] = vw
    return mat

# Формат входных данных для MapReduce
def INPUTFORMAT():
    first_mat = [((0, i, j), small_mat[i, j]) for i in range(small_mat.shape[0]) for j in range(small_mat.shape[1])]
    yield first_mat

    second_mat = [((1, j, k), big_mat[j, k]) for j in range(big_mat.shape[0]) for k in range(big_mat.shape[1])]
    yield second_mat

# Фаза Map для умножения матриц (разделение по ключу)
def MAP_JOIN(k1, v1):
    (mat_num, i, j) = k1
    w = v1
    if mat_num == 0:
        yield (j, (mat_num, i, w))  # Отправляем элемент первой матрицы с ключом j
    else:
        yield (i, (mat_num, j, w))  # Отправляем элемент второй матрицы с ключом i

# Фаза Reduce для соединения значений по ключам
def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])  # Умножаем соответствующие элементы

# Получение результатов объединения
def GET_JOINED():
    for j in joined:
        print("aa", j)
        yield j[1]

# Фаза Map для суммирования произведений
def MAP_MUL(k1, v1):
    yield (k1, v1)

# Фаза Reduce для суммирования элементов
def REDUCE_MUL(key, values):
    yield (key, sum(values))  # Складываем все элементы с одинаковым ключом

# Количество карт и редьюсеров
maps = 4
reducers = 2

# Выполняем первый этап MapReduce (умножение матриц)
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполняем второй этап MapReduce (суммирование)
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

# Преобразуем результат в окончательный список
solution = [v for p in pre_result for v in p[1]]
print(solution)

# Сравниваем с эталонным решением
np.allclose(reference_solution, asmatrix(solution))


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.5409688396516846), ((0, 1), 0.441185527701592), ((0, 2), 0.4910468588748887), ((0, 3), 0.4801939333926947), ((0, 4), 0.6950785530075944), ((0, 5), 0.588347636140214), ((0, 6), 0.5204682823169958), ((0, 7), 0.03346097225018539), ((0, 8), 0.061217039746889676), ((0, 9), 0.13793607495149351), ((0, 10), 0.14942634958329193), ((0, 11), 0.07471615324234389), ((0, 12), 0.5818660446490205), ((0, 13), 0.4873849685529308), ((0, 14), 0.20507872772684926), ((0, 15), 0.6130156788504186), ((0, 16), 0.5057341253934137), ((0, 17), 0.029269564402916882), ((0, 18), 0.42354084309763174), ((0, 19), 0.3529006750426488), ((0, 20), 0.20641716200374066), ((0, 21), 0.34781947974095917), ((0, 22), 0.39261276602193473), ((0, 23), 0.2770538148416106), ((0, 24), 0.5761187812850077), ((0, 25), 0.1822449061379207), ((0, 26), 0.44774034585808437), ((0, 27), 0.6729133102378071), ((0, 28), 0.09247388883381122), ((0, 29), 0.634149765537203), ((0, 30), 0.2176

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
# Определение размеров матриц
I = 2  # Количество строк первой матрицы и строк в итоговой матрице
J = 3  # Количество столбцов первой матрицы и строк второй матрицы
K = 4*10  # Количество столбцов второй матрицы и столбцов в итоговой матрице

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Генерация первой матрицы
big_mat = np.random.rand(J, K)  # Генерация второй матрицы

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():

  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.2838202541303115), ((0, 1), 0.1381410394140817), ((0, 2), 0.2774778820554505), ((0, 3), 0.44759213567133715), ((0, 4), 0.3277371368240059), ((0, 5), 0.41269877350304696), ((0, 6), 0.14212818367980362), ((0, 7), 0.40495316556192773), ((0, 8), 0.11648499674352768), ((0, 9), 0.01964134414232802), ((0, 10), 0.5545778383431961), ((0, 11), 0.48008136642785454), ((0, 12), 0.26123163021718354), ((0, 13), 0.3125302622068577), ((0, 14), 0.2870375361988848), ((0, 15), 0.26784467509945087), ((0, 16), 0.5182267894839125), ((0, 17), 0.15003975786544882), ((0, 18), 0.28924179262432503), ((0, 19), 0.48919192086778246), ((0, 20), 0.6303093268706532), ((0, 21), 0.4238259018538415), ((0, 22), 0.13505412274748563), ((0, 23), 0.24080313067569836), ((0, 24), 0.4876947633971493), ((0, 25), 0.35048821048890794), ((0, 26), 0.364454279951223), ((0, 27), 0.695607132412629), ((0, 28), 0.3703318372705892), ((0, 29), 0.03339466198160071), ((0, 30), 0.55

True